# Generate Rank-Rank Hypergeometric Overlap (RRHO) plots

In [ ]:
library(RRHO)
library(dplyr)
library(lattice)

In [ ]:
get_deg_df <- function(sex, tissue){
    if(tolower(sex) == 'female'){
        fn = paste0("../../../", tolower(tissue),"/female_analysis/_m/genes/diffExpr_szVctl_full.txt")
    } else {
        fn = paste0("../../../", tolower(tissue),"/male_analysis/_m/genes/diffExpr_szVctl_full.txt")
    }
    if(tolower(tissue) == 'cmc_dlpfc'){
        df = data.table::fread(fn) %>% mutate(metric=-log10(P.Value)*sign(t), gencodeID=gene_id) %>% 
            select('gencodeID', 'metric')
    } else {
        df = data.table::fread(fn) %>% mutate(metric=-log10(P.Value)*sign(t)) %>% 
            select('gencodeID', 'metric')
    }
    return(df)
}


within_tissue_rrho_plot <- function(tissue){
    defaultW <- getOption("warn") 
    options(warn = -1) 
    f.list = get_deg_df('female', tolower(tissue)) %>% distinct(gencodeID, .keep_all = TRUE) 
    m.list = get_deg_df('male', tolower(tissue)) %>% distinct(gencodeID, .keep_all = TRUE) 
    rrho_df <- RRHO(f.list, m.list, BY=TRUE, labels=c("Female", "Male"), 
                    alternative='enrichment', log10.ind=TRUE)
    rrho_df$hypermat[is.na(rrho_df$hypermat)] <- 0 # for NA, correct to 0
    options(warn = defaultW)
    plt = levelplot(rrho_df$hypermat, col.regions=viridis::viridis(100), 
                    xlab="Female", ylab="Male", at=seq(0, 1000, 100))
    print(plt)
    label = paste("rrho_sex_compare", tolower(tissue), sep='_')
    R.devices::devEval(c("pdf", "png"), name=label, print(plt))
    #return(rrho_df)
}


## Within tissue comparison

In [ ]:
for(tissue in c('caudate', 'dlpfc', 'hippocampus', 'cmc_dlpfc')){
    within_tissue_rrho_plot(tissue)
}

## Female comparison across tissues

In [ ]:
compare_tissue_plot <- function(sex, tissue1, tissue2){
    list1 = get_deg_df(sex, tissue1) %>% distinct(gencodeID, .keep_all = TRUE) 
    list2 = get_deg_df(sex, tissue2) %>% distinct(gencodeID, .keep_all = TRUE) 
    gene_overlap = intersect(list1$gencodeID, list2$gencodeID)
    list1 = list1 %>% filter(gencodeID %in% gene_overlap)
    list2 = list2 %>% filter(gencodeID %in% gene_overlap)
    rrho_df <- RRHO(list1, list2, BY=TRUE, labels=c(tissue1, tissue2), 
                   alternative='enrichment', log10.ind=TRUE)
    rrho_df$hypermat[is.na(rrho_df$hypermat)] <- 0 # for NA, correct to 0
    rrho_plot = levelplot(rrho_df$hypermat, col.regions=viridis::viridis, 
                          xlab=tissue1, ylab=tissue2, at=seq(0, 800, 100))
    R.devices::devEval(c("pdf", "png"), 
                       name=paste("rrho",sex,tissue1,tissue2, sep='_'), 
                       print(rrho_plot))
}

### Female

In [ ]:
compare_tissue_plot("female", "Caudate", "DLPFC")
compare_tissue_plot("female", "Caudate", "Hippocampus")
compare_tissue_plot("female", "DLPFC", "Hippocampus")

In [ ]:
compare_tissue_plot("female", "CMC_DLPFC", "Caudate")
compare_tissue_plot("female", "CMC_DLPFC", "DLPFC")
compare_tissue_plot("female", "CMC_DLPFC", "Hippocampus")

### Male

In [ ]:
compare_tissue_plot("male", "Caudate", "DLPFC")
compare_tissue_plot("male", "Caudate", "Hippocampus")
compare_tissue_plot("male", "DLPFC", "Hippocampus")

In [ ]:
compare_tissue_plot("male", "CMC_DLPFC", "Caudate")
compare_tissue_plot("male", "CMC_DLPFC", "DLPFC")
compare_tissue_plot("male", "CMC_DLPFC", "Hippocampus")

## Repreducibility Information

In [ ]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()